# Summarize Results and Generate the Table

In [1]:
import polars as pl
import pandas as pd

## Define func

In [2]:
ROUND_NDIGITS = 3

def process_dataframe(df: pl.DataFrame, round_ndigits: int) -> pl.DataFrame:
    df = df.with_columns(
        pl.when(pl.col("ai_name") == "mix")
        .then(pl.col("num_ai") * 3)
        .otherwise(pl.col("num_ai"))
        .alias("num_ai_processed")
    )

    def format_mean_only(col_name):
            return (
                pl.col(col_name)
                .map_elements(
                    lambda x: f"{x:.{round_ndigits}f}" if x is not None else "", 
                    return_dtype=pl.String
                )
                .str.replace(r"^0\.", ".")  
            )

    df_formatted = df.with_columns([
        format_mean_only("accuracy_mean").alias("Accuracy"),
        format_mean_only("recall_complaint_mean").alias("Recall")
    ])

    df_pivot = df_formatted.pivot(
        index=["ai_name", "method"],
        columns="num_ai_processed",
        values=["Accuracy", "Recall"],
        aggregate_function="first"
    )

    df_pivot = df_pivot.with_columns(
        pl.when(pl.col("method").str.starts_with("DS")).then(0)
        .when(pl.col("method").str.starts_with("HSDS")).then(2)
        .otherwise(1)
        .alias("_sort_rank")
    ).sort(["ai_name", "_sort_rank", "method"]).drop("_sort_rank")

    value_cols = [c for c in df_pivot.columns if c not in ["ai_name", "method"]]
    
    def sort_key(col_name):
        metric, num = col_name.rsplit("_", 1)
        num = int(num)
        metric_order = 0 if metric == "Accuracy" else 1
        return (num, metric_order)

    sorted_cols = sorted(value_cols, key=sort_key)
    
    return df_pivot.select(["ai_name", "method"] + sorted_cols)

## Load Data

In [3]:
df = pl.read_csv("ds1task5_aggregation_results_crowd_kit_stan.csv", truncate_ragged_lines=True)
tmp_df = pl.read_csv("ds1task5_aggregation_results_cbcc.csv", truncate_ragged_lines=True)
df = pl.concat([df, tmp_df])
tmp_df = pl.read_csv("ds1task5_aggregation_results_truth_infer.csv", truncate_ragged_lines=True)
df = pl.concat([df, tmp_df])
tmp_df = pl.read_csv("ds1task5_aggregation_results_minmax.csv", truncate_ragged_lines=True)
df = pl.concat([df, tmp_df])

df = df.filter(
    (
        pl.col("num_ai").is_in([0,3,6,9]) & (pl.col("ai_name") != pl.lit("mix"))
    ) | (pl.col("num_ai").is_in([0,1,2,3]) & (pl.col("ai_name") == pl.lit("mix")))
)

## Data Preprocessing

In [4]:
## 
group_keys = ['ai_name', 'num_ai', 'method']
ignore_col = 'iter'
all_cols = df.columns
target_cols = [c for c in all_cols if c not in group_keys + [ignore_col]]
uc_cols = [c for c in target_cols if c.startswith('uc_')]
special_method = "HSDS_MCMC(iter_sampling=3000)"
df_std = df.filter(pl.col("method") != special_method)
df_special = df.filter(pl.col("method") == special_method)
res_std = df_std.group_by(group_keys).agg([
    pl.col(target_cols).mean().name.suffix("_mean"),
    pl.col(target_cols).std().name.suffix("_std")
])
if df_special.height > 0:
    df_special = df_special.with_columns(
        pl.sum_horizontal(uc_cols).alias("uc_total_sum")
    )
    df_special_filtered = df_special.filter(
        pl.col("uc_total_sum") == pl.col("uc_total_sum").min().over(group_keys)
    )
    res_special = df_special_filtered.group_by(group_keys).agg([
        pl.col(target_cols).mean().name.suffix("_mean"),
        *[pl.lit(None).cast(pl.Float64).alias(f"{c}_std") for c in target_cols]
    ])
else:
    res_special = res_std.clear()
result = pl.concat([res_std, res_special], how="vertical")
result = result.sort(group_keys)

In [5]:
df2 = process_dataframe(result, 2)

/tmp/ipykernel_2551/2333177843.py:26: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  df_pivot = df_formatted.pivot(


In [6]:
df2

ai_name,method,Accuracy_0,Recall_0,Accuracy_3,Recall_3,Accuracy_6,Recall_6,Accuracy_9,Recall_9
str,str,str,str,str,str,str,str,str,str
"""f100""","""BDS(iter_sampling=3000)""",""".87""",""".60""",""".82""",""".37""",""".82""",""".37""",""".82""",""".37"""
"""f100""","""CATD""",""".83""",""".39""",""".82""",""".37""",""".82""",""".37""",""".82""",""".37"""
"""f100""","""CBCC_M=2""",""".86""",""".87""",""".82""",""".37""",""".82""",""".37""",""".82""",""".37"""
"""f100""","""CBCC_M=4""",""".86""",""".87""",""".82""",""".37""",""".82""",""".37""",""".82""",""".37"""
"""f100""","""CBCC_M=8""",""".86""",""".87""",""".82""",""".37""",""".82""",""".37""",""".82""",""".37"""
…,…,…,…,…,…,…,…,…,…
"""mix""","""PM-CRH""",""".82""",""".59""",""".84""",""".62""",""".86""",""".60""",""".83""",""".38"""
"""mix""","""ZC""",""".84""",""".42""",""".85""",""".40""",""".86""",""".42""",""".84""",""".37"""
"""mix""","""minmax""",""".73""",""".00""",""".90""",""".73""",""".91""",""".77""",""".87""",""".54"""


In [7]:
df2.write_csv("ds1task5_aggregation_results_all_processed.csv")